# Image recognition: MNIST

First of all, let's do some imports:

In [1]:
from __future__ import print_function
import numpy as np
import sys
import os
import cntk

To feed data to the network, we create a data reader - `MinibatchSource`. In our case input data are text files, so we use `CTFDeserializer` to parse them. In case of real images, we use `ImageDeserializer`.

Data Readers can take care of:
 * Applying filters to the data on the fly (image resize, crop, etc.)
 * Data Augmentation
 * Cycling over the data as many times as needed
 * Data shuffling

In [2]:
def create_reader(path, is_training, input_dim, label_dim):
    return cntk.io.MinibatchSource(cntk.io.CTFDeserializer(path, cntk.io.StreamDefs(
        features  = cntk.io.StreamDef(field='features', shape=input_dim),
        labels    = cntk.io.StreamDef(field='labels',   shape=label_dim)
    )), randomize=is_training, max_sweeps = cntk.io.INFINITELY_REPEAT if is_training else 1)

Now let's define some network parameters and CNN network architecture. In this code, `z` represents the computation graph for our neural network, and `input_var` and `label_var` are the data which we feed to the network during training.

In [3]:
image_height = 28
image_width  = 28
num_channels = 1
input_dim = image_height * image_width * num_channels
num_output_classes = 10

# Input variables denoting the features and label data
input_var = cntk.ops.input((num_channels, image_height, image_width), np.float32)
label_var = cntk.ops.input(num_output_classes, np.float32)

# Instantiate the feedforward classification model
scaled_input = cntk.ops.element_times(cntk.ops.constant(0.00390625), input_var)

with cntk.layers.default_options(activation=cntk.ops.relu, pad=False): 
    conv1 = cntk.layers.Convolution2D((5,5), 32, pad=True)(scaled_input)
    pool1 = cntk.layers.MaxPooling((3,3), (2,2))(conv1)
    conv2 = cntk.layers.Convolution2D((3,3), 48)(pool1)
    pool2 = cntk.layers.MaxPooling((3,3), (2,2))(conv2)
    conv3 = cntk.layers.Convolution2D((3,3), 64)(pool2)
    f4    = cntk.layers.Dense(96)(conv3)
    drop4 = cntk.layers.Dropout(0.5)(f4)
    z     = cntk.layers.Dense(num_output_classes, activation=None)(drop4)

Next, we define loss function to minimize, and some training parameters and objects. `trainer` is the main object that is used for training.

In [4]:
ce = cntk.losses.cross_entropy_with_softmax(z, label_var)
pe = cntk.metrics.classification_error(z, label_var)

reader_train = create_reader('Train-28x28_cntk_text.txt', True, input_dim, num_output_classes)

# Training config
epoch_size = 60000                    
minibatch_size = 64
max_epochs = 10

# Set learning parameters
lr_per_sample    = [0.001]*10 + [0.0005]*10 + [0.0001]
lr_schedule      = cntk.learning_rate_schedule(lr_per_sample, cntk.learners.UnitType.sample, epoch_size)
mm_time_constant = [0]*5 + [1024]
mm_schedule      = cntk.learners.momentum_as_time_constant_schedule(mm_time_constant, epoch_size)

# Instantiate the trainer object to drive the model training
learner = cntk.learners.momentum_sgd(z.parameters, lr_schedule, mm_schedule)
progress_printer = cntk.logging.ProgressPrinter(tag='Training',
                                                num_epochs=max_epochs)
trainer = cntk.Trainer(z, (ce, pe), learner, progress_printer)

Here actual training happens. `input_map` specifies correspondence between data in input reader and variables. Then we loop through all samples and call `trainer.train_minibatch`

In [5]:
# Define mapping from reader streams to network inputs
input_map = {
    input_var : reader_train.streams.features,
    label_var : reader_train.streams.labels
}

cntk.logging.log_number_of_parameters(z) ; print()

# Get minibatches of images to train with and perform model training
for epoch in range(max_epochs):       # loop over epochs
    sample_count = 0
    while sample_count < epoch_size:  # loop over minibatches in the epoch
        data = reader_train.next_minibatch(min(minibatch_size, epoch_size - sample_count), input_map=input_map) # fetch minibatch.
        trainer.train_minibatch(data)                                   # update model with it
        sample_count += data[label_var].num_samples                     # count samples processed so far

    trainer.summarize_training_progress()
    # z.save(os.path.join(model_path, "ConvNet_MNIST_{}.dnn".format(epoch)))
    

Training 98778 parameters in 10 parameter tensors.

Learning rate per 1 samples: 0.001
Momentum per 1 samples: 0.0
Finished Epoch[1 of 10]: [Training] loss = 0.404131 * 60000, metric = 12.96% * 60000 21.392s (2804.8 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.104888 * 60000, metric = 3.00% * 60000 3.573s (16792.6 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.077668 * 60000, metric = 2.24% * 60000 3.605s (16643.6 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.063216 * 60000, metric = 1.84% * 60000 3.575s (16783.2 samples/s);
Finished Epoch[5 of 10]: [Training] loss = 0.054544 * 60000, metric = 1.60% * 60000 3.582s (16750.4 samples/s);
Momentum per 1 samples: 0.999023914182
Finished Epoch[6 of 10]: [Training] loss = 0.046677 * 60000, metric = 1.36% * 60000 3.590s (16713.1 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.040269 * 60000, metric = 1.21% * 60000 3.553s (16887.1 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.037695 * 60000, 

Now let's test it on test data:

In [6]:
# Load test data
reader_test = create_reader('Test-28x28_cntk_text.txt', False, input_dim, num_output_classes)

input_map = {
    input_var : reader_test.streams.features,
    label_var : reader_test.streams.labels
}

# Test data for trained model
epoch_size = 10000
minibatch_size = 1024

# Process minibatches and evaluate the model
metric_numer    = 0
metric_denom    = 0
sample_count    = 0
minibatch_index = 0

while sample_count < epoch_size:
    current_minibatch = min(minibatch_size, epoch_size - sample_count)

    # Fetch next test min batch.
    data = reader_test.next_minibatch(current_minibatch, input_map=input_map)

    # Minibatch data to be trained with
    metric_numer += trainer.test_minibatch(data) * current_minibatch
    metric_denom += current_minibatch

    # Keep track of the number of samples processed so far.
    sample_count += data[label_var].num_samples
    minibatch_index += 1

print("")
print("Final Results: Minibatch[1-{}]: errs = {:0.2f}% * {}".format(minibatch_index+1, (metric_numer*100.0)/metric_denom, metric_denom))
print("")


Final Results: Minibatch[1-11]: errs = 0.83% * 10000

